In [14]:
import json
data_file=open("D:\\ml_ve\\Dataset\\freego2.json").read()
#print(data_file)

In [34]:
intents=json.loads(data_file)
# print(intents)

In [16]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

words=[]
document=[]
classes=[]
p=[]

ignore_words=["$","(",")",";","]","[",","]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        p.append(pattern)
        #print(pattern)
        w=nltk.word_tokenize(pattern)
        words.extend(w)
        document.append((w,intent['responses']))
        classes.append(intent['responses'])
# print(classes)
# print(len(classes))
# print(words)
# print(len(words))
# print(document)
# print(len(document))
# print(len(p))

In [17]:
# Remove stopwords that is defined and the lemmetize the word
words=[lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
# print(words)
# print(len(words))

In [18]:
# merge the duplicate and sort them
words=sorted(list(set(words)))
# print(words)
# print(len(words))

In [19]:
# merge the duplicate and sort them
classes=sorted(list(set(classes)))
# print(len(classes))
# print(classes)

In [20]:
print("no: of words:",len(words))
print("no: of documents:",len(document))
print("no: of classes:",len(classes))

no: of words: 304
no: of documents: 195
no: of classes: 53


In [21]:
import pickle
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [22]:
# create training data
training=[]
output_empty=[0]*len(classes)
# print(output_empty)

In [23]:
for doc in document:
    bag=[]
    pattern_words=doc[0]
#     print(pattern_words)
    pattern_words=[lemmatizer.lemmatize(w.lower()) for w in pattern_words]
#     print(pattern_words)
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
#     print(bag)
    output_row=list(output_empty)
#     print(output_row)
    output_row[classes.index(doc[1])]=1
#     print(output_row)
    training.append([bag,output_row])
#     print(training)   

In [24]:
import numpy as np
import random
random.shuffle(training)
training=np.array(training)
# print(training)
# print(training.shape)

<ipython-input-24-db16201fa09e>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training=np.array(training)


In [26]:
# create input and response data from training data
train_x=list(training[:,0])
train_y=list(training[:,1])
# print(len(train_y[0]))
# print(len(train_x[0]))
# print(train_x)
# print(train_y)

In [13]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
from keras.optimizers import SGD

model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

sgd=SGD(learning_rate=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)
model.save('chatbot_model.h5',hist)
print("model created")

Epoch 1/200
39/39 [==============================] - 20s 1ms/step - loss: 3.9788 - accuracy: 0.0051  
Epoch 2/200
39/39 [==============================] - 0s 1ms/step - loss: 3.9524 - accuracy: 0.0315
Epoch 3/200
39/39 [==============================] - 0s 2ms/step - loss: 3.9024 - accuracy: 0.0532
Epoch 4/200
39/39 [==============================] - 0s 2ms/step - loss: 3.8517 - accuracy: 0.1016
Epoch 5/200
39/39 [==============================] - 0s 2ms/step - loss: 3.8006 - accuracy: 0.0588
Epoch 6/200
39/39 [==============================] - 0s 2ms/step - loss: 3.7421 - accuracy: 0.0844
Epoch 7/200
39/39 [==============================] - 0s 1ms/step - loss: 3.6896 - accuracy: 0.1088
Epoch 8/200
39/39 [==============================] - 0s 1ms/step - loss: 3.5418 - accuracy: 0.1710
Epoch 9/200
39/39 [==============================] - 0s 1ms/step - loss: 3.5655 - accuracy: 0.1967
Epoch 10/200
39/39 [==============================] - 0s 1ms/step - loss: 3.3923 - accuracy: 0.1545
Epoch 

39/39 [==============================] - 0s 1ms/step - loss: 0.1303 - accuracy: 0.9447
Epoch 163/200
39/39 [==============================] - 0s 1ms/step - loss: 0.2036 - accuracy: 0.9346
Epoch 164/200
39/39 [==============================] - 0s 1ms/step - loss: 0.2899 - accuracy: 0.9036
Epoch 165/200
39/39 [==============================] - 0s 971us/step - loss: 0.2218 - accuracy: 0.9201
Epoch 166/200
39/39 [==============================] - 0s 919us/step - loss: 0.2168 - accuracy: 0.9069
Epoch 167/200
39/39 [==============================] - 0s 971us/step - loss: 0.1602 - accuracy: 0.9589
Epoch 168/200
39/39 [==============================] - 0s 866us/step - loss: 0.1901 - accuracy: 0.9384
Epoch 169/200
39/39 [==============================] - 0s 971us/step - loss: 0.1449 - accuracy: 0.9569
Epoch 170/200
39/39 [==============================] - 0s 997us/step - loss: 0.2256 - accuracy: 0.9132
Epoch 171/200
39/39 [==============================] - 0s 1ms/step - loss: 0.1854 - accuracy:

In [27]:
# load the file that has been saved
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open("D:\\ml_ve\\Dataset\\freego2.json").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [28]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [29]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [30]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    results = res.argmax()
    return_list = []
    return_list.append({"intent": classes[results]})
    return return_list

In [31]:
def getResponse(ints, intents_json,text):
    Res = ints[0]['intent']
    list_of_intents = intents_json['intents']
    t=[]
    t.append(text)
    for i in list_of_intents:
        result=[]
        if(i['responses']== Res) and (i['patterns']==t):
            result.append({"Q": i['responses'], "O": i['options'], "A": i['answer']})
            break
    return result

In [32]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents,text)
    return res

In [ ]:
for i in p:
    pred=pr

In [42]:
import tkinter
from tkinter import *
from PIL import ImageTk,Image

root=tkinter.Tk()
root.title("FreegoQuiz")
root.geometry("700x600")
root.config(background="#ffffff")
# root.resizable(0,0)

user_answers=[]
answer=[]
indexes=[]
Q=1

def indexgen():
    global indexes
    for i in range(3,13):
        indexes.append(i)

def backend(score):
    labelquestion.destroy()
    r1.destroy()
    r2.destroy()
    r3.destroy()
    r4.destroy()

    labelheading=Label(root,text="Your Result",font=("comic sans MS",18,"bold"),background="#ffffff")
    labelheading.pack()
    
    img3=ImageTk.PhotoImage(Image.open("passed.png"))
    img4=ImageTk.PhotoImage(Image.open("failed.png"))
    
    labelresultimg=Label(root)
    labelresultimg.pack(pady=(100,0))
    
    labelresulttext=Label(root,font=("Consolas",18,"bold"))
    labelresulttext.pack(pady=(0,10))
    
    if(score>=40):
        labelresultimg.configure(image=img3,background="#ffffff")
        labelresultimg.image=img3
        labelresulttext.configure(text="Congragulation!!!\nYou have Passed",background="#ffffff")
    else:
        labelresultimg.configure(image=img4,background="#ffffff")
        labelresultimg.image=img4
        labelresulttext.configure(text="Sorry!!!\nYou have Failed\nBetter Luck next Time",background="#ffffff")
    
    labelscore=Label(root,text=("Total Score:",score),font=("Consolas",14))
    labelscore.pack(pady=(10,0))

def calScore(user_answers,answer):
    global indexes
    score=0
    x=3
    for i in indexes:
        if user_answers[x]==answer[i]:
            score+=5
        x+=1
    backend(score)


def selected():
    global Q
    global qst,opt,answer,ans
    global radiovar,ques
    global labelquestion,r1,r2,r3,r4
    x=radiovar.get()
    user_answers.append(x)
    if(Q<13):
        res=chatbot_response(x)
        qst=res[0]['Q']
        opt=res[0]['O']
        ans=res[0]['A']
        answer.extend(ans)
        ques="Q:) " + qst
        labelquestion.config(text=ques)
        r1.config(text=opt[0],value=opt[0]) 
        r2.config(text=opt[1],value=opt[1])
        r3.config(text=opt[2],value=opt[2])
        r4.config(text=opt[3],value=opt[3])
        Q+=1
    else:
        calScore(user_answers,answer)
        
        
    
def stratquiz():
    global labelquestion,r1,r2,r3,r4
    global radiovar,ques
    s="Start"
    res=chatbot_response(s)
    qst=res[0]['Q']
    opt=res[0]['O']
    ans=res[0]['A']
    ques="Q:) " + qst
    answer.extend(ans)

    labelquestion=Label(root,text=ques,font=("Consolas",14),width=500,justify="center",wraplength=400)
    labelquestion.pack(pady=(100,50))

    radiovar = StringVar()
    radiovar.set(-1)

    r1=Radiobutton(root,text=opt[0],font=("Times",12),value=opt[0],variable=radiovar,justify="center",background="#ffffff",command=selected)
    r1.pack(pady=(0,10))

    r2=Radiobutton(root,text=opt[1],font=("Times",12),value=opt[1],variable=radiovar,justify="center",background="#ffffff",command=selected)
    r2.pack(pady=(0,10))

    r3=Radiobutton(root,text=opt[2],font=("Times",12),value=opt[2],variable=radiovar,justify="center",background="#ffffff",command=selected)
    r3.pack(pady=(0,10))

    r4=Radiobutton(root,text=opt[3],font=("Times",12),value=opt[3],variable=radiovar,justify="center",background="#ffffff",command=selected)
    r4.pack(pady=(0,10))

def start():
    labelimage.destroy()
    labelinstruction.destroy()
    labeltext.destroy()
    btnstart.destroy()
    labelrules.destroy()
    indexgen()
    stratquiz()


img1=ImageTk.PhotoImage(Image.open("freego.png"))
labelimage=Label(root,image=img1,background="#ffffff")
labelimage.pack(pady=(50,0))

labeltext=Label(root,text="Interview",font=("comic sans MS",18,"bold"),background="#ffffff")
labeltext.pack(pady=(0,60))

img2=ImageTk.PhotoImage(Image.open("start.png"))

btnstart=Button(root,image=img2,relief=FLAT,border=0,background="#ffffff",command=start)
btnstart.pack()

labelinstruction=Label(root,text="Read the Rules below And\nClick Start once You are Ready",background="#ffffff",font=("Consolas",10),justify="center")
labelinstruction.pack(pady=(0,150))

labelrules=Label(root,text="1. Read the questions clearly and answer\n2. Once the option is selected it trigger next question so select the option wisely\n3. You only have one chance for each questions\n4. There is no negative score for wrong answer",width=150,background="#000000",foreground="#ffffff",font=("Times",13))
labelrules.pack()

root.mainloop()